In [ ]:
# ОБРАЗЕЦ 

class DataPipeline:
    """Подготовка исходных данных"""
    
    def __init__(self):
        """Параметры класса"""
        self.medians = None
        self.longitude_median = None
        self.latitude_median = None
        
    def fit(self, df):
        """Сохранение статистик"""
        
        # Расчет медиан
        self.medians = df[['population', 'housing_median_age', 'total_bedrooms']].median()
        self.longitude_median = df['longitude'].median()
        self.latitude_median = df['latitude'].median()
        
    def transform(self, df):
        """Трансформация данных"""
        
        # 1. Пропуски
        df[['population', 'housing_median_age', 'total_bedrooms']] =\
            df[['population', 'housing_median_age', 'total_bedrooms']].fillna(self.medians)
        
        
        df.replace(
            {'ocean_proximity':
                {'-': df['ocean_proximity'].mode()[0]}
            }, inplace=True)
        
        # 2. Выбросы (outliers)
        df.loc[df['longitude'] > 0, 'longitude'] = df.loc[df['longitude'] > 0, 'longitude'] * -1
        df.loc[df['longitude'] == 0, 'longitude'] = self.longitude_median
        df.loc[(df['latitude'] <= 0) | (df['latitude'] > 50), 'latitude'] = self.latitude_median
        
        
        # 3. Новые фичи (features)
        
        # Доля спален в общем кол-ве комнат
        df['bedroom_share'] = df['total_bedrooms'] / df['total_rooms'] * 100

        # Сколько человек в среднем живут в одной комнате
        df['population_per_room'] = df['population'] / df['total_rooms']
        
        # 4. Обработка категорий
        df = pd.concat([df, pd.get_dummies(df['ocean_proximity'])], axis=1)
        
        return df


In [ ]:
# убираем пропуски
def clear_empty_values(data):
    # смотрим пустые значения
    empty_values = data.isna().sum()
    for i, j in empty_values.items():
        # отбираем столбы с пропусками
        if j > 0:
            # если количество пропусков больше 40%, то такой столбец удаляем
            if j / train_data.shape[0] > 0.3:
                # значения убираем
                train_data.drop(i, axis=1, inplace=True)
            else:
                # остальное заменяем на медиану
                train_data[i] = train_data[i].fillna(train_data[i].median())
    return data